### to test function written in cytnx

In [1]:
import numpy as np
from scipy import linalg
import cytnx as cy 
import matplotlib.pyplot as plt
import pandas as pd


def mappingDtype2Cytnx(dpp):
    mapping = {'float':cy.Type.Float,
               'int':cy.Type.Int32,
               'complex':cy.Type.ComplexFloat,
               'bool':cy.Type.Bool}
    try:
        return mapping[dpp]
    except KeyError:
        print("no key {} inside mapping dtype function".format(dpp))

In [3]:
N = 2
beta = 1
dpp = 'float'
Cydtype = mappingDtype2Cytnx(dpp)

        
weightMatrix = np.array([[np.exp(beta),np.exp(-1*beta)],
                                 [np.exp(-1*beta),np.exp(beta)]])
weightMatrix = cy.from_numpy(weightMatrix)
#print(weightMatrix)
pureNode = cy.zeros((N,N,N,N),dtype=Cydtype)
pureNode[0,0,0,0] = pureNode[1,1,1,1] = 1
impureNode = cy.zeros((N,N,N,N),dtype=Cydtype)
impureNode[0,0,0,0] = 1;impureNode[1,1,1,1] = -1
Uni_pureNode = cy.UniTensor([cy.Bond(2),cy.Bond(2),cy.Bond(2),cy.Bond(2)],rowrank=2)
Uni_pureNode.put_block(pureNode)
Uni_impureNode = cy.UniTensor([cy.Bond(2),cy.Bond(2),cy.Bond(2),cy.Bond(2)],rowrank=2)
Uni_impureNode.put_block(impureNode)
vals,vecs = cy.linalg.Eig(weightMatrix)



In [9]:
Test  = cy.UniTensor([cy.Bond(2),cy.Bond(3),cy.Bond(4),cy.Bond(5)],rowrank=2)
# odd random.Make_normal didn't return anything, but still need to put back to original unitensor
A = Test.get_block()
cy.random.Make_normal(A,0,0.1)
print(type(A))
Test.put_block(A)
Test.set_labels([-1,-2,1,2])
Test2 = Test.permute([0,1,2,0]) # should it be a bug? it should give us a warrning.
Test2.print_diagram()
# Test = Test.permute([2,3,1,0])  # bug ? should be labels but not original index 

Test.print_diagram()
# help(Test.permute)

<class 'cytnx.cytnx.Tensor'>
-----------------------
tensor Name : 
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
    -1 ____| 2         4 |____ 1  
           |             |     
    -2 ____| 3         2 |____ -1 
           \             /     
            -------------      
-----------------------
tensor Name : 
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
    -1 ____| 2         4 |____ 1  
           |             |     
    -2 ____| 3         5 |____ 2  
           \             /     
            -------------      


In [7]:
A = Test.get_block()
cy.random.Make_normal(A,0,1)
Test.put_block(A)
T_conj = Test.Conj()
Test.set_labels([0,-1,1,-2])
T_conj.set_labels([2,-1,3,-2])
T_conj.print_diagram()
Result = Test.contract(T_conj)

# Result = Result.permute([0,1,2,3],rowrank=2)
Result.print_diagram()

-----------------------
tensor Name : 
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     2 ____| 2         4 |____ 3  
           |             |     
    -1 ____| 3         5 |____ -2 
           \             /     
            -------------      
-----------------------
tensor Name : 
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 4         4 |____ 3  
           \             /     
            -------------      


In [8]:
AA = Result.get_block()
AA = AA.reshape(4,4)
print(AA)
print(type(AA))

RuntimeError: 
# Cytnx error occur at void cytnx::Tensor_impl::reshape_(const std::vector<long int>&)
# error: [ERROR] new shape does not match the number of elements.
# file : /home/travis/miniconda/envs/test-environment/conda-bld/cytnx_1600323034980/work/include/Tensor.hpp (232)


Total elem: 16
type  : Double (Float64)
cytnx device: CPU
Shape : (2,2,2,2)
[[[[9.76915e+00 2.67467e+00 ]
   [-1.11613e+00 3.49892e+00 ]]
  [[2.67467e+00 9.93828e-01 ]
   [1.34970e-01 1.04591e+00 ]]]
 [[[-1.11613e+00 1.34970e-01 ]
   [3.25440e+00 -1.04046e+00 ]]
  [[3.49892e+00 1.04591e+00 ]
   [-1.04046e+00 3.64861e+00 ]]]]



